# Коронавирус

Для выполнения задач этого проекта вам потребуется весь полученный ранее опыт. А применять его мы будем к теме, которая будоражит умы человечества с начала 2020 года: мы будем анализировать данные о заражениях *COVID-19*.

Вы будете работать с данными, актуальными на 18 сентября 2020 года. Для начала [скачайте данные](https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.json) из источника и сохраните их в файл `data/covid.json`.

На самом верхнем уровне находится словарь, в котором ключ — это код страны. В значении лежат данные по стране. Полное название страны лежит в поле location. Кроме поля location в данных есть несколько полей с общей статистикой:

|       Показатель      |              Описание             |
|:---------------------:|:---------------------------------:|
|      "population"     |         количество жителей        |
|  "population_density" |        плотность населения        |
|      "median_age"     |         медианный возраст         |
|    "aged_65_older"    |      доля людей старше 65 лет     |
|    "aged_70_older"    |      доля людей старше 70 лет     |
| "diabetes_prevalence" |       доля людей с диабетом       |
|   "life_expectancy"   | ожидаемая продолжительность жизни |

…а также некоторые другие статистики.

> Имейте в виду, что по некоторым странам есть неполные данные.

Кроме того, в данном объекте есть поле *data* с собственно данными по заражениям. Оно указывает на массив (*list*) с данными, каждый элемент которого содержит следующие поля:

|   Показатель   |           Описание          |
|:--------------:|:---------------------------:|
|     "date"     | дата (в формате YYYY-MM-DD) |
|  "total_cases" |   общее количество случаев  |
|   "new_cases"  |     новые случаи за день    |
| "total_deaths" |   общее количество смертей  |
|  "new_deaths"  |     новые смерти за день    |
|  "total_tests" |   общее количество тестов   |
|   "new_tests"  |     новые тесты за день     |

Для зарегистрированных случаев и для количества смертей также указываются данные количества, нормированные на миллион населения (например, *new_cases_per_million*). Для тестов указывается количество тестов на тысячу населения (например, *new_tests_per_thousand*).

> Имейте в виду, что не все страны передают все данные, поэтому какие-то поля могут отсутствовать.

In [1]:
!curl "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.json" > data/covid.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 34.2M  100 34.2M    0     0  85.0M      0 --:--:-- --:--:-- --:--:-- 85.0M


In [2]:
import json
import math

In [3]:
data = json.load(open('data/covid.json', 'rt'))

## Задание 1

Результат: напишите функцию `max_value(value_name)`, которая вернёт название страны, у которой показатель `value_name` максимален.

Давайте для начала найдём страны с максимальными показателями. На вход вашей функции придёт параметр `value_name`. Вам нужно вернуть `location` страны, у которой показатель `value_name` максимален.

- Sample input: `max_value('median_age')`
- Sample output: `"Japan"`

In [4]:
def max_value(value_name: str) -> str:
    xs = [(x['location'], x.get(value_name, 0))
          for x in data.values()
    ]
    m = max(xs, key=lambda x: x[1])
    return m[0]

In [5]:
max_value('median_age')

'Japan'

# Задание 2

Результат: напишите функцию `day_with_max_value(value_name)`, которая вернёт дату в августе (дата должна иметь вид `"2020-08-DD"`). Суммарный показатель `value_name` по всем странам в эту дату должен быть максимален (наш проверочный тест передаст в качестве `value_name` один из показателей поля `data`: например, `new_tests`, `total_deaths` и так далее).

Теперь посмотрим на дневные показатели. Давайте найдём день в августе, в который сумма показателя `value_name` по всем странам была максимальна. Напишите функцию `day_with_max_value(value_name)`, которая вернёт строку, обозначающую этот день.

- Sample input: `day_with_max_value("new_cases")`
- Sample output: `"2020-08-13"`

In [6]:
def is_aug(record) -> bool:
    date = record['date']
    y, m, d = date.split('-')
    return y == '2020' and m == '08'

In [7]:
def day_with_max_value(value_name: str) -> str:
    result = None
    max_value = 0
    for country in data.values():
        for record in country['data']:
            value = record.get(value_name, 0)
            date = record['date']
            if is_aug(record) and value > max_value:
                result = date
                max_value = value
    return result

In [8]:
day_with_max_value('new_cases')

'2020-08-13'

## Задание 3

Результат: реализуйте функцию `correlation_cases_tests(country_code)`, возвращающую корреляцию между количеством тестов и количеством найденных случаев в августе.

Теперь давайте посчитаем коэффициент корреляции между количеством тестов и количеством обнаруженных случаев.

Вы можете использовать следующую функцию для её вычисления:

In [9]:
def corr(data_x, data_y):
    sx, sy, sxy, sx2, sy2, n = 0.0, 0.0, 0.0, 0.0, 0.0, 0
    for x, y in zip(data_x, data_y):
        sx += x
        sx2 += x * x
        sy += y
        sy2 += y * y
        sxy += x * y
        n += 1
    dd = math.sqrt((sx2 / n - (sx / n * sx / n)) * (sy2 / n - (sy / n * sy / n)))
    if abs(dd) < 1e-5:
        return None
    return (sxy / n - sx / n * sy / n) / dd

Этот коэффициент показывает, насколько зависимы сравниваемые величины. Если при росте одной наблюдается рост второй, то коэффициент будет близок к 1; если при росте одной наблюдается падение второй, то к -1. Если же зависимость не наблюдается, то коэффициент будет близок к 0.

Если посчитать этот коэффициент для тестов и обнаруженных случаев, то можно получить несколько идей о происходящих процессах. Например, если корреляция отрицательная, это может говорить о том, что во время спада эпидемии страна резко нарастила количество тестов. Если корреляция высокая, то, вероятно, до какого-то момента тестов не хватало, а с ростом тестирования начала расти выявляемость.

Если вы хотите сделать уверенные выводы, вам стоит смотреть на эти величины в динамике и принимать во внимание все факторы. Сама по себе корреляция ни о чём не говорит, но может натолкнуть вас на какие-то идеи.

Напишите функцию `correlation_cases_tests(country_code)`, которая вернёт значение коэффициента корреляции между количеством тестов и количеством найденных случаев в августе. Если в какой-то день один из показателей отсутствует, возьмите его значение равным 0.

- Sample input: `correlation_cases_tests("IND")`
- Sample output: `0.5935340285658485`

In [10]:
def correlation_cases_tests(country_code: str) -> float:
    country = data[country_code]

    xs = [(x.get('new_cases', 0), x.get('new_tests', 0))
        for x in country['data']
        if is_aug(x)
    ]
    cases = [c for c, t in xs]
    tests = [t for c, t in xs]

    return corr(cases, tests) 

In [11]:
correlation_cases_tests('IND')

0.5935340285658485

## Задание 4

Результат: реализуйте функцию `sigma(country_code, value_name)`.

Мы хотим определить, насколько стабилен параметр `value_name` (например, количество зарегистрированных случаев). Для этого мы можем вычислить среднеквадратическое отклонение — корень из дисперсии. Вы уже сталкивались с этим показателем, когда выполняли задания по случайным величинам.

Напомним, среднее и СКО можно посчитать так:

In [12]:
def calc_mean_sigma(data):
    sx, sx2, n = 0.0, 0.0, 0
    for x in data:
        sx += x
        sx2 += x * x
        n += 1
    return (sx / n, math.sqrt(max(sx2 / n - sx / n * sx / n, 0)))

Этот параметр показывает, насколько сильно «разбросан» показатель вокруг своего среднего значения.

Как и в предыдущей задаче, это значение может дать вам некоторые идеи о происходящих процессах. Если разброс очень большой, то это может говорить о бурно развивающемся процессе. Возможно и другое объяснение: некоторые страны не публикуют статистику по выходным, а в начале недели, наоборот, возникают скачки. Если вы посмотрите на то, как СКО меняется со временем, вы можете выдвинуть некоторые гипотезы и попытаться их подтвердить.

Ваша функция `sigma` должна принимать два параметра — `country_code` и `value_name`, и должна возвращать одно число — среднеквадратическое отклонение величины `value_name` за август.

- Sample input: `sigma("GBR", "new_cases")`
- Sample output: `235.76391655560022`

In [13]:
def sigma(country_code: str, value_name: str) -> float:
    country = data[country_code]
    xs = [x.get(value_name, 0)
        for x in country['data']
        if is_aug(x)
    ]

    return calc_mean_sigma(xs)[1]

In [14]:
sigma('GBR', 'new_cases')

235.76391655560022

## Задание 5

Результат: реализуйте функцию `sigma_norm(country_code, value_name)`.

В предыдущей задаче мы вычислили среднеквадратическое отклонение. Для разных стран этот показатель сложно сравнивать, так как средние значения показателей в разных странах отличаются.

Давайте попробуем нормировать СКО на матожидание величины. Формулы для вычисления СКО и среднего приведены в предыдущей задаче.

На какие идеи вас могут натолкнуть вычисленные коэффициенты? Если матожидание достаточно большое, а дисперсия мала, это может говорить о стабилизировавшемся процессе и идеальной системе учёта случаев. А может, наоборот, о том, что данные искусственно помещаются в заданный коридор. Попробуйте посчитать нормированное СКО для разных показателей.

Ваша функция `sigma_norm` должна принимать два параметра — `country_code` и `value_name`, и должна возвращать одно число — среднеквадратическое отклонение, разделённое на матожидание величины `value_name` в августе.

- Sample input: `sigma_norm("RUS", "new_cases")`
- Sample output: `0.05748491556302192`

In [15]:
def sigma_norm(country_code: str, value_name: str) -> float:
    country = data[country_code]
    xs = [x.get(value_name, 0)
        for x in country['data']
        if is_aug(x)
    ]
    mean, sigma = calc_mean_sigma(xs)
    return sigma / mean

In [16]:
sigma_norm('RUS', 'new_cases')

0.05748491556302192